In [90]:
import fastjet as fj
import Nsubjettiness as Nsub
import numpy as np
import pandas as pd

In [91]:
fn = "events_LHCO2020_BlackBox2.h5"
chunksize = 1000
# chunksize=1
total_size = 10**6
# total_size=1
def generator(filename, chunksize=chunksize,total_size=total_size):

    i = 0
    
    while True:

        yield pd.read_hdf(filename,start=i*chunksize, stop=(i+1)*chunksize).values

        i+=1
        if (i+1)*chunksize > total_size:
            break
            

gen = generator(fn)

In [92]:
tau21 = Nsub.NsubjettinessRatio(2,1,Nsub.OnePass_WTA_KT_Axes(),Nsub.UnnormalizedMeasure(1.))
tau32 = Nsub.NsubjettinessRatio(3,2,Nsub.OnePass_WTA_KT_Axes(),Nsub.UnnormalizedMeasure(1.))
tau43 = Nsub.NsubjettinessRatio(4,3,Nsub.OnePass_WTA_KT_Axes(),Nsub.UnnormalizedMeasure(1.))

In [93]:
#Now, let's cluster some jets!

outdata=np.zeros((total_size,11))
jet_def = fj.JetDefinition(fj.antikt_algorithm, 1.0)

for i,events in enumerate(gen):
    if (i%10==0):
        print(i*chunksize)
    pass
    for j,event in enumerate(events):


        numparticles = np.sum(event[::3]>0)
        particles = []
        
        for k in range(numparticles):
            particles += [fj.PtYPhiM(event[k*3],event[k*3+1],event[k*3+2],0)]

        sequence = fj.ClusterSequence(particles,jet_def)
        jets = fj.sorted_by_pt(sequence.inclusive_jets(ptmin=20))
        mjj = (jets[0]+jets[1]).m()
        m1 = jets[0].m()
        m2 = jets[1].m()
        tau21_1 = tau21.result(jets[0])
        tau21_2 = tau21.result(jets[1])
        tau32_1 = tau32.result(jets[0])
        tau32_2 = tau32.result(jets[1])
        tau43_1 = tau32.result(jets[0])
        tau43_2 = tau32.result(jets[1])
        npart_1 = len(jets[0].constituents())
        npart_2 = len(jets[1].constituents())
        outdata[i*chunksize + j] = np.array([mjj,m1,m2,tau21_1,tau21_2,
                                            tau32_1,tau32_2,
                                            tau43_1,tau43_2,
                                            npart_1,npart_2])


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000


In [101]:
outstore = pd.HDFStore("/home/jack/projects/CWoLa-Hunting/LHCO_data/BB2_features.h5")
columns = ["mjj","mj_1","mj_2","tau21_1","tau21_2","tau32_1","tau32_2","tau43_1","tau43_2","ntrk_1","ntrk_2"]
dictionary = dict(zip(columns, outdata.T))
out_df = pd.DataFrame(dictionary,columns=columns)
outstore.append('jetfeatures',out_df)
outstore.close()

In [96]:
out_df 

,mjj,mj_1,mj_2,tau21_1,tau21_2,tau32_1,tau32_2,tau43_1,tau43_2,ntrk_1,ntrk_2
0,2096.183412,223.069712,67.170832,0.579873,0.579036,0.638075,0.526968,0.638075,0.526968,81.0,40.0
1,2397.087083,242.136131,77.530168,0.231106,0.789789,0.611108,0.806549,0.611108,0.806549,30.0,38.0
2,3707.091130,95.748156,583.321056,0.736647,0.180463,0.735905,0.589461,0.735905,0.589461,24.0,44.0
3,4376.529474,440.656938,144.897147,0.695671,0.628274,0.430099,0.618668,0.430099,0.618668,63.0,28.0
4,3062.404935,51.966746,116.588300,0.204460,0.438050,0.410539,0.691067,0.410539,0.691067,8.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...
999995,3933.177701,333.614314,124.994022,0.713052,0.298036,0.761013,0.751069,0.761013,0.751069,104.0,39.0
999996,3027.998325,77.524902,420.259124,0.599249,0.194313,0.576260,0.770836,0.576260,0.770836,33.0,58.0
999997,3198.569075,632.781059,189.418110,0.209839,0.705933,0.455122,0.759474,0.455122,0.759474,76.0,65.0
999998,3638.008043,100.477660,150.893778,0.275823,0.709033,0.800408,0.481507,0.800408,0.481507,23.0,44.0


In [78]:
jets[0].m()

103.83178590787475

In [56]:
(jets[0]+jets[1]).m()

2096.183412215967

In [48]:
alljets

array([[[ 1.23468857e+03, -6.28262735e+02,  1.16933234e+00,
          1.25467818e+03],
        [ 8.68532203e+02,  3.70963072e+02, -1.11802349e+02,
          8.78270957e+02],
        [ 1.41415077e+02,  1.10611717e+02, -4.87225457e+02,
          5.09794281e+02],
        [ 1.11158096e+02,  6.96153746e+01,  8.96182010e+02,
          9.03168559e+02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]])

In [12]:
fj.JetDefinition(fj.antikt_algorithm, 0.5)

<fastjet.JetDefinition; proxy of <Swig Object of type 'fastjet::JetDefinition *' at 0x7fd1459c57e0> >

In [36]:
jets[0].pt()

1202.5259775628438

In [37]:
jets[0].px()

-1024.7741288829075

In [55]:
len(jets[1].constituents())

40

In [60]:
jets[1].constituents()[20].pt()

35.6268196105957